In [1]:
import os

In [2]:
%pwd

'd:\\Text-Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig :

    root_dir:Path 
    data_path: Path
    tokenizer_name: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories



In [7]:
class ConfigurationManager:

    def __init__(
        
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name =config.tokenizer_name,
             
        )

        return data_transformation_config    

In [9]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset,load_from_disk

In [10]:
class DataTransformation:

    def __init__(self, config: DataTransformationConfig):

        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_to_features(self,example):

        input_encodings = self.tokenizer(example['dialogue'], max_length = 1024,truncation=True)

        with self.tokenizer.as_target_tokenizer():

            target_encodings = self.tokenizer(example['summary'],max_length=128)

        return{
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels' : target_encodings['input_ids']
        }
    

    def convert(self):

        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_to_features,batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,'samsum_dataset'))


  

In [12]:
try:
    config= ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config= data_transformation_config)
    data_transformation.convert()


except Exception as e :
    raise e 



[2023-10-24 16:53:39,067: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-24 16:53:39,070: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-24 16:53:39,073: INFO: common: created directory at: artifacts]
[2023-10-24 16:53:39,076: INFO: common: created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]d:\Text-Summarizer\textsumvenv\lib\site-packages\transformers\tokenization_utils_base.py:3848: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 54269.00 examples/s]
